In [1]:
import pandas as pd

if 'NCBI_INFO' not in globals():
    print("Reading gene2refseq.gz...")
    NCBI_INFO = pd.read_csv("../Data/ncbi/gene2refseq.gz", sep='\t', compression='gzip')
else:
    print("NCBI_INFO already loaded.")


Reading gene2refseq.gz...


C:\Users\richa\AppData\Local\Temp\ipykernel_18148\2687780475.py:5: DtypeWarning: Columns (2,4,6,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  NCBI_INFO = pd.read_csv("../Data/ncbi/gene2refseq.gz", sep='\t', compression='gzip')


In [2]:
import numpy as np
import json
from scipy.sparse import load_npz
import os
from tqdm import tqdm
FOLDER = "DGIDB_FULL_CANCER/"
DGIDB_DIRECTORY = "../Gen_Hypergraph/output/" + FOLDER
DGIDB_CONVERGED_VECTOR_PATH = "./output/" + FOLDER + "DGIDB_vector.npy"
MSIGDB_DIRECTORY = "../Gen_Hypergraph/output/MSigDB_FULL/"
OUTPUT_FOLDER = "./output/" + FOLDER
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
restart_prob = 0.2  # Restart probability (theta)
num_iterations = 10  # Number of iterationsh


In [3]:
# Open the JSON file and load its content into a dictionary
with open(DGIDB_DIRECTORY + "gene_to_index.json", "r") as file:
    dgidb = json.load(file)
with open(MSIGDB_DIRECTORY + "gene_to_index.json", "r") as file:
    msigdb = json.load(file)

FileNotFoundError: [Errno 2] No such file or directory: '../Gen_Hypergraph/output/DGIDB_FULL_CANCER/gene_to_index.json'

In [ ]:
import numpy as np
# Jump probability for matching genes
w = 1

# Number of genes (assuming they are both of same size or matchable)
num_genes_dgidb = len(dgidb)
num_genes_msigdb = len(msigdb)

# Initialize the inter-layer matrix (D) with zeros
D = np.zeros((num_genes_dgidb, num_genes_msigdb))
i = 0
# Build the inter-layer matrix (D)
for gene_dgidb, idx_dgidb in dgidb.items():
    # If the gene exists in both gene-to-index mappings
    if gene_dgidb in msigdb:      
        idx_msigdb = msigdb[gene_dgidb]
        D[idx_dgidb, idx_msigdb] = w  # Set jump probability
        i += 1
rows_with_high_sum = np.where(D.sum(axis=1) > 0)[0]

In [ ]:
# Load matrices
MSIGDB_weighted_matrix = load_npz(MSIGDB_DIRECTORY + "hypergraph_incidence_matrix_weighted.npz")
MSIGDB_binary_matrix = load_npz(MSIGDB_DIRECTORY + "hypergraph_incidence_matrix_binary.npz")
DGIDB_binary_matrix = load_npz(DGIDB_DIRECTORY + "hypergraph_incidence_matrix_binary.npz")
DGIDB_vector =  np.load(DGIDB_CONVERGED_VECTOR_PATH)

num_genes_MSIGDB = MSIGDB_binary_matrix.shape[0]  # Number of genes in MSIGDB
# Initialize probability vectors
v0 = np.zeros(num_genes_MSIGDB)  # Combined vector for DGIDB and MSIGDB
teleport = np.zeros(num_genes_MSIGDB)  # Restart probability vector
# Initialize probability vectors
v0[:] = 1.0 / (num_genes_MSIGDB)  # Starting on DGIDB
teleport[:] = 1.0 / (num_genes_MSIGDB)

def get_hyper_randomwalk(DGIDB_binary_matrix, DGIDB_vector, MSIGDB_weighted_matrix, MSIGDB_binary_matrix, D, restart_prob, num_iterations):
    """Performs a hypergraph-based random walk with restart considering inter-layer transitions."""
    vi = v0.copy()  # Start with uniform probability (combined for both DGIDB and MSIGDB)
    distance_list = []
    # Initialize the complete vector with zeros
    dgidb_vector_complete = np.zeros(num_genes_MSIGDB)

    for gene in tqdm(range(num_genes_MSIGDB), desc="DGIDB full vector calculation"):
        # Intra-hypergraph transitions in MSIGDB (moving within MSIGDB)
        connected_pathways = MSIGDB_binary_matrix[gene, :].nonzero()[1]  # Nonzero columns in MSIGDB binary matrix

        if len(connected_pathways) == 0:
            continue  # Skip if no pathways are found

        for pathway in connected_pathways:
            # Find genes connected to the selected pathway (weighted transition in MSIGDB)
            connected_genes = MSIGDB_weighted_matrix[:, pathway].toarray().flatten()
            neighbor_genes = np.where(connected_genes > 0)[0]  # Get genes with nonzero weight

            # Check if the current gene has a DGIDB connection
            dgidb_gene = np.where(D[:, gene] > 0)[0]  # Find DGIDB neighbors of the current MSIGDB gene
            if len(dgidb_gene) == 1:
                dgidb_drugs = DGIDB_binary_matrix[dgidb_gene[0], :].nonzero()[1]
                neighbor_genes_set = set()  # To avoid duplicates
            
                for drug in dgidb_drugs:
                    # Get genes connected through the same drug (edge)
                    connected_genes = DGIDB_binary_matrix[:, drug].toarray().flatten()
                    neighbor_genes = np.where(connected_genes > 0)[0]
                    # Add unique neighbors to the set
                    neighbor_genes_set.update(neighbor_genes)

                # Sum contributions from unique DGIDB neighbors
                if len(neighbor_genes_set) > 0:
                    neighbor_genes_list = list(neighbor_genes_set)
                    dgidb_contribution = np.sum(DGIDB_vector[neighbor_genes_list])  # Sum unique contributions
                    dgidb_vector_complete[gene] += dgidb_contribution  # Store in the complete vector

    # Normalize dgidb_vector_complete to avoid overflow
    dgidb_vector_complete /= np.sum(dgidb_vector_complete) if np.sum(dgidb_vector_complete) > 0 else 1
    


    for k in range(num_iterations):
        print(f"{k+1} iteration")

        # Store previous probability vector
        vj = vi.copy()

        # Initialize new probability vector
        vi_new = np.zeros_like(vi)
        # Handle transitions for MSIGDB (second part of the vector)
        for gene in tqdm(range(num_genes_MSIGDB), desc="MSIGDB Gene Processing"):
            # Intra-hypergraph transitions in MSIGDB (moving within MSIGDB)
            connected_pathways = MSIGDB_binary_matrix[gene, :].nonzero()[1]  # Nonzero columns in MSIGDB binary matrix

            if len(connected_pathways) == 0:
                continue  # Skip if no pathways are found

            # Collect probabilities from neighbors in MSIGDB
            prob_sum = 0
            for pathway in connected_pathways:
                # Find genes connected to the selected pathway (weighted transition in MSIGDB)
                connected_genes = MSIGDB_weighted_matrix[:, pathway].toarray().flatten()
                neighbor_genes = np.where(connected_genes > 0)[0]  # Get genes with nonzero weight

                if len(neighbor_genes) == 0:
                    continue

                # Normalize weights within MSIGDB
                weights = connected_genes[neighbor_genes]
                weight_sum = np.sum(weights)
                if weight_sum > 0:
                    weights /= weight_sum  # Normalize to sum to 1

                # Transition probability contribution
                prob_sum += np.sum(weights * vj[neighbor_genes])


            # Add inter-hypergraph jump probabilities (from MSIGDB to DGIDB)
            vi_new[gene] = prob_sum
        print("vi_new", vi_new)
        # Normalize vi_new to avoid overflow
        vi_new /= np.sum(vi_new) if np.sum(vi_new) > 0 else 1

        # Apply restart probability
        vi = restart_prob * vi_new + (1 - restart_prob) * teleport + dgidb_vector_complete

        # Calculate distance (convergence criterion)
        distance = np.sum(np.abs(vj - vi))
        distance_list.append(distance)

    # Sort importance scores in descending order
    importance_scores = np.argsort(vi)[::-1]
    importance_values = vi[importance_scores]

    # Return importance scores and distance values
    return {"Importance": list(zip(importance_scores, importance_values)), "Distance": distance_list}

In [ ]:
import torch
from tqdm import tqdm

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Convert sparse matrices to dense (or use PyTorch sparse support if memory limited)
MSIGDB_weighted = torch.tensor(MSIGDB_weighted_matrix.toarray(), dtype=torch.float32, device=device)
MSIGDB_binary = torch.tensor(MSIGDB_binary_matrix.toarray(), dtype=torch.float32, device=device)
DGIDB_binary = torch.tensor(DGIDB_binary_matrix.toarray(), dtype=torch.float32, device=device)
# DGIDB_vector = torch.tensor(DGIDB_vector, dtype=torch.float32, device=device)
# D = torch.tensor(D, dtype=torch.float32, device=device)

num_genes_MSIGDB = MSIGDB_binary.shape[0]
v0 = torch.full((num_genes_MSIGDB,), 1.0 / num_genes_MSIGDB, dtype=torch.float32, device=device)
teleport = v0.clone()
P = MSIGDB_weighted @ MSIGDB_weighted.T


Using device: cuda


In [ ]:

def get_hyper_randomwalk_torch(DGIDB_binary_matrix, DGIDB_vector, MSIGDB_weighted_matrix, MSIGDB_binary_matrix, D, restart_prob, num_iterations, P):
    vi = v0.clone()
    distance_list = []
    dgidb_vector_complete = np.zeros(num_genes_MSIGDB)

    for gene in tqdm(range(num_genes_MSIGDB), desc="DGIDB full vector calculation"):
        # Intra-hypergraph transitions in MSIGDB (moving within MSIGDB)
        connected_pathways = MSIGDB_binary_matrix[gene, :].nonzero()[1]  # Nonzero columns in MSIGDB binary matrix

        if len(connected_pathways) == 0:
            continue  # Skip if no pathways are found

        for pathway in connected_pathways:
            # Find genes connected to the selected pathway (weighted transition in MSIGDB)
            connected_genes = MSIGDB_weighted_matrix[:, pathway].toarray().flatten()
            neighbor_genes = np.where(connected_genes > 0)[0]  # Get genes with nonzero weight

            # Check if the current gene has a DGIDB connection
            dgidb_gene = np.where(D[:, gene] > 0)[0]  # Find DGIDB neighbors of the current MSIGDB gene
            if len(dgidb_gene) == 1:
                dgidb_drugs = DGIDB_binary_matrix[dgidb_gene[0], :].nonzero()[1]
                neighbor_genes_set = set()  # To avoid duplicates
            
                for drug in dgidb_drugs:
                    # Get genes connected through the same drug (edge)
                    connected_genes = DGIDB_binary_matrix[:, drug].toarray().flatten()
                    neighbor_genes = np.where(connected_genes > 0)[0]
                    # Add unique neighbors to the set
                    neighbor_genes_set.update(neighbor_genes)

                # Sum contributions from unique DGIDB neighbors
                if len(neighbor_genes_set) > 0:
                    neighbor_genes_list = list(neighbor_genes_set)
                    dgidb_contribution = np.sum(DGIDB_vector[neighbor_genes_list])  # Sum unique contributions
                    dgidb_vector_complete[gene] += dgidb_contribution  # Store in the complete vector

    # Normalize dgidb_vector_complete to avoid overflow
    dgidb_vector_complete /= np.sum(dgidb_vector_complete) if np.sum(dgidb_vector_complete) > 0 else 1
    dgidb_vector_complete = torch.from_numpy(dgidb_vector_complete).to(device)
    for k in range(num_iterations):
        print(f"Iteration {k + 1}")
        vj = vi.clone()

        # Vectorized transition step
        # Resulting shape will be (n, n)

        # Step 2: Normalize P by dividing by the sum of each column (with a small constant to avoid division by zero)
        P = P / (P.sum(dim=0, keepdim=True) + 1e-9)
        vi_new = torch.matmul(P.T.float(), vj.float())  # Transition from vj to vi_new using matrix multiplication

        # Normalize and combine
        vi_new = vi_new / (vi_new.sum() + 1e-9)
        
        vi = restart_prob * vi_new.T + (1 - restart_prob) * teleport + dgidb_vector_complete

        distance = torch.sum(torch.abs(vj - vi)).item()
        distance_list.append(distance)


    importance_scores = torch.argsort(vi, descending=True)
    importance_values = vi[importance_scores]

    return {
        "Importance": list(zip(importance_scores.tolist(), importance_values.tolist())),
        "Distance": distance_list
    }


In [ ]:
# result = get_hyper_randomwalk(DGIDB_binary_matrix, DGIDB_vector, MSIGDB_weighted_matrix, MSIGDB_binary_matrix, D, restart_prob, num_iterations)
# Print results
result = get_hyper_randomwalk_torch(DGIDB_binary_matrix, DGIDB_vector, MSIGDB_weighted_matrix, MSIGDB_binary_matrix, D, restart_prob, num_iterations, P)
print("Top Indices by Importance:")
for index, score in result["Importance"][:10]:
    print(f"Index {index}: {score:.6f}")

print("\nDistance per Iteration:")
print(result["Distance"])

DGIDB full vector calculation: 100%|██████████| 21981/21981 [33:42<00:00, 10.87it/s]  


Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Top Indices by Importance:
Index 13090: 0.037095
Index 17490: 0.033613
Index 15579: 0.020103
Index 10310: 0.014310
Index 15588: 0.012995
Index 9287: 0.012799
Index 15373: 0.011371
Index 17766: 0.011349
Index 17061: 0.011099
Index 1265: 0.010304

Distance per Iteration:
[1.0000007301168807, 0.05653582025115611, 0.0008196470116672572, 1.72265681612771e-05, 4.4657281250692904e-07, 5.8542354963719845e-08, 5.727270036004484e-08, 5.479523679241538e-08, 5.4675183491781354e-08, 5.659967428073287e-08]


In [ ]:
# Load the JSON data from the file
with open(MSIGDB_DIRECTORY + 'gene_to_index.json', 'r') as file:
    gene_to_index = json.load(file)

# Invert the dictionary to map indices back to genes
index_to_gene = {v: k for k, v in gene_to_index.items()}
human_gene2refseq = NCBI_INFO[NCBI_INFO['#tax_id'] == 9606]
id_to_gene_claim = pd.Series(human_gene2refseq.Symbol.values, index=human_gene2refseq.GeneID).to_dict()

def get_gene_claim_name(ncbi_gene_id):
    ncbi_gene_id = int(ncbi_gene_id)
    result = id_to_gene_claim[ncbi_gene_id]
    if result:
        return result
    else:
        return "Gene name not found"

In [ ]:
results_df = pd.DataFrame(result["Importance"], columns=['Index', 'Score'])
results_df["ncbi_gene_id"] = results_df["Index"].apply(index_to_gene.get)
results_df["claim_name"] = results_df["ncbi_gene_id"].apply(get_gene_claim_name)
results_df.to_csv(OUTPUT_FOLDER + "unidirectional_multilayer_rwr_results.csv", index=False)